In [20]:
import io
from pypdf import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# 1. Register the Armenian font
pdfmetrics.registerFont(
    TTFont('NotoSansArmenian', 'NotoSansArmenian-Black.ttf')
)

def add_footer_with_arm_text_and_two_links(
    input_path: str,
    output_path: str,
    prefix: str,
    icon_path: str,
    links: list[tuple[str, str]],
    connector: str = " և ",  # Armenian “and”
    font_name: str = "NotoSansArmenian",
    font_size: int = 10,
    margin_bottom_mm: float = 12,
    icon_text_gap_pt: float = 5
):
    """
    prefix: leading Armenian text before the icon (e.g. "Խնդրում ենք դիտել ")
    arm_text: Armenian display text after icon (if needed)
    icon_path: path to your 'camera.png'
    links: exactly two (display_text, url) tuples
    connector: text between the two links (default Armenian “ և ”)
    """
    if len(links) != 2:
        raise ValueError("Must provide exactly two links")

    icon = ImageReader(icon_path)
    reader = PdfReader(input_path)
    writer = PdfWriter()

    for page in reader.pages:
        w = float(page.mediabox.width)
        h = float(page.mediabox.height)

        packet = io.BytesIO()
        can = canvas.Canvas(packet, pagesize=(w, h))
        can.setFont(font_name, font_size)

        # Measure widths for centering
        prefix_w    = can.stringWidth(prefix, font_name, font_size)
        icon_size   = font_size
        first_text, first_url   = links[0]
        second_text, second_url = links[1]
        first_w     = can.stringWidth(first_text, font_name, font_size)
        connector_w = can.stringWidth(connector, font_name, font_size)
        second_w    = can.stringWidth(second_text, font_name, font_size)

        total_w = prefix_w + icon_size + icon_text_gap_pt + first_w + connector_w + second_w
        x = (w - total_w) / 2
        y = margin_bottom_mm * mm

        # Draw Armenian prefix
        can.drawString(x, y, prefix)
        x += prefix_w

        # Draw camera icon
        can.drawImage(icon,
                      x,
                      y - (font_size * 0.1),
                      width=icon_size,
                      height=icon_size,
                      mask='auto')
        x += icon_size + icon_text_gap_pt

        # Draw & link first Armenian/Latin text
        can.drawString(x, y, first_text)
        can.linkURL(first_url, (x, y, x + first_w, y + font_size), relative=0)
        x += first_w

        # Draw Armenian connector
        can.drawString(x, y, connector)
        x += connector_w

        # Draw & link second text
        can.drawString(x, y, second_text)
        can.linkURL(second_url, (x, y, x + second_w, y + font_size), relative=0)

        can.save()
        packet.seek(0)

        overlay = PdfReader(packet)
        page.merge_page(overlay.pages[0])
        writer.add_page(page)

    with open(output_path, "wb") as f_out:
        writer.write(f_out)


if __name__ == "__main__":
    add_footer_with_arm_text_and_two_links(
        input_path="input.pdf",
        output_path="output_with_arm_footer.pdf",
        prefix="Կարանք լինքեր դնենք՝",
        icon_path="camera.jpg",
        links=[
            ("էս մեկ", "https://www.youtube.com/watch?v=y4qtOVq_e0U"),
            ("էս էլ երկու", "https://www.youtube.com/watch?v=6_2ZJ4QW_O4")
        ]
    )


https://chatgpt.com/share/e/6851f7fa-1554-8012-89ab-38852aee2d45

In [3]:
!pip install pypdf reportlab


   ---------------------------------------- 304.2/304.2 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 2.0/2.0 MB 1.5 MB/s eta 0:00:00
